In [ ]:
data_dir = "/data/projects/dschaub/ANCA-GN_transcriptomics/data/single-cell/exploratory"
working_dir = "/data/projects/dschaub/ANCA-GN_transcriptomics"

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
os.chdir(working_dir)

import yaml
import anndata as ad
import matplotlib.pyplot as plt
import mudata as md
import muon as mu
import numpy as np
import scanpy as sc
import scanpy.external as sce
from scipy import sparse
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap

from utils.utils import *
from utils.plotting import *

sc.settings.verbosity = 0
sc.set_figure_params(dpi=80)
sns.set(style="white")

## Load data

In [ ]:
path = os.path.join(
    data_dir, "ANCA_exploratory_27PK27PB_Tcells_HarmonyR_annotated.h5mu"
)
mudata = md.read_h5mu(path)
mod_rna = mudata.mod["rna"]
mod_cite = mudata.mod["cite"]
mudata

In [ ]:
len(mod_rna.obs["patient"].unique())

### Subset CD4EMRM

In [ ]:
celltype_key = "cell_type"

In [ ]:
mudata.mod["rna"].obs[celltype_key].unique()

In [ ]:
mudata = mudata[mudata.mod["rna"].obs[celltype_key] == "CD4+ EM/RM"].copy()
mudata

In [ ]:
mod_rna = mudata.mod["rna"]
mod_cite = mudata.mod["cite"]

## Filter data

In [ ]:
print(mod_rna.shape)
sc.pp.filter_genes(mod_rna, min_cells=10)
print(mod_rna.shape)

### Quality control

In [ ]:
plot_qc(mod_rna, celltype_key)

In [ ]:
# number of unique patients
len(mod_rna.obs["patient"].unique())

## Integrate data

In [ ]:
log_normalize(mod_rna)
protein_clr(mod_cite)

In [ ]:
model, rna_subset = integrate_with_totalvi(
    mod_rna,
    mod_cite,
    batch_key="patient",
    n_top_genes=4000,
    # continuous_covariate_keys=["pct_counts_mt", "total_counts"],
    # continuous_covariate_keys=["total_counts"],
    # continuous_covariate_batch_keys=["pct_counts_mt"],
    empirical_protein_background_prior=True,
)

In [ ]:
plt.plot(model.history["elbo_train"], label="train")
plt.plot(model.history["elbo_validation"], label="val")
plt.title("Negative ELBO over training epochs")
# plt.ylim(500, 1500)
plt.legend()

In [ ]:
rep_name = "X_totalvi"
mod_rna.obsm[rep_name] = model.get_latent_representation()
sc.pp.neighbors(mod_rna, use_rep=rep_name)
sc.tl.umap(mod_rna)

In [ ]:
sc.pl.umap(
    mod_rna,
    color=["sample", "patient", "tissue"],
    # return_fig=True,
    # legend_loc="on data",
    # save="_leiden.png",
    wspace=0.8,
)

In [ ]:
resolutions = np.linspace(0.1, 2.0, 20)
for res in resolutions:
    res = np.round(res, 1)
    print(f"Running Leiden clustering with resolution {res}")
    sc.tl.leiden(mod_rna, key_added=f"leiden_{res}", resolution=res)

In [ ]:
plot_leiden_results(mod_rna, rep_name="")

## Save data

In [ ]:
save_path = os.path.join(data_dir, "ANCA_exploratory_27PK27PB_CD4Teff_TotalVI.h5mu")
mudata = md.MuData(
    {
        "rna": mod_rna.copy(),
        "cite": mod_cite.copy(),
    }
)
mudata.write_h5mu(save_path)